In [1]:
from keras.preprocessing.sequence import TimeseriesGenerator
from FC_RNN_Evaluater.FC_RNN_Evaluater import *
from keras.initializers import RandomNormal
import numpy as np

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
timesteps = 10
inputMatrix = np.random.rand(57,224,224,3)# np.array([[[[i, i, i]]] for i in range(57)])
labels = np.array([[i, i, i] for i in range(57)])

inputMatrix, inputLabels, outputLabels = getSequencesToSequences(inputMatrix, labels, timesteps)
batch_size=1    
img_gen = TimeseriesGenerator(inputMatrix, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
ang_gen = TimeseriesGenerator(inputLabels, outputLabels, length=timesteps, sampling_rate=1, stride=timesteps, batch_size=batch_size)
batch_01 = img_gen[0]
batch_0 = ang_gen[0]
inputMatrix, y = batch_01
inputLabels, outputLabels = batch_0

In [3]:
from FC_RNN_Evaluater.Stateful_FC_RNN_Configuration import *
from FC_RNN_Evaluater.FC_RNN_Evaluater import *
from FC_RNN_Evaluater.ReinforceAlgorithmForKerasModels import *

In [4]:
vgg_model, full_model, modelID, preprocess_input = getFinalModel(timesteps = timesteps, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout, 
                      num_outputs = num_outputs, lr = learning_rate, include_vgg_top = include_vgg_top, use_vgg16 = use_vgg16)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as k
from keras import losses
import numpy as np
import tensorflow as tf

In [8]:
im = tf.convert_to_tensor(inputMatrix, name='im', dtype=tf.float32)
il = tf.convert_to_tensor(inputLabels, name='il', dtype=tf.float32)

In [9]:
y = full_model([im, il])

In [10]:
y

<tf.Tensor 'model_2_1/time_distributed_1/transpose_1:0' shape=(1, ?, 3) dtype=float32>

In [14]:
trainingBiwi = readBIWIDataset(subjectList = trainingSubjects, preprocess_input = preprocess_input)

All frames and annotations from 1 datasets have been read by 2019-02-20 01:22:41.366291


In [76]:
def drawSamples(model, episodes, sigma, outputs, seed=None):
    samplesShape = outputs.shape
    #samplesShape = ((episodes,)+outputs.shape)
    print(samplesShape, type(samplesShape))
    distribution = RandomNormal(mean=model.outputs, stddev=sigma, seed=seed)
    return distribution(samplesShape)

def getGradientsPerEpisode(model, samples, targets):
    gradients_per_episode = []
    rewards = getRewardsWithBaselinePerEpisode(samples, targets)
    for i in range(samples.shape[0]):
        loss = losses.mean_squared_error(targets, samples[i])
        gradients = k.gradients(loss, model.trainable_weights)
        rewardedGradients = [g*rewards[i] for g in gradients]
        gradients_per_episode.append(rewardedGradients)
    return gradients_per_episode, tf.reduce_mean(rewards, axis=0)

def getFinalGradients(model, samples, targets):
    gradients_per_episode, reward = getGradientsPerEpisode(model, samples, targets)
    stacked_gradients = []
    for i in range(len(gradients_per_episode[0])):
        stacked_gradients.append(tf.stack([gradients[i] for gradients in gradients_per_episode])) 
    return [tf.reduce_mean(g, axis=0) for g in stacked_gradients], reward


In [77]:
def getOutputTensor(model, inputMatrix_batch, inputLabels_batch):
    im = tf.convert_to_tensor(inputMatrix_batch, name='im', dtype=tf.float32)
    il = tf.convert_to_tensor(inputLabels_batch, name='il', dtype=tf.float32)
    return model([im, il])

In [78]:
def reinforceModelForEpoch(model, data_gen, episodes, sigma, steps_per_epoch, epochCount, verbose=1):
    i = 0
    if verbose == 1: printProgressBar(0, steps_per_epoch, prefix = epochCount, suffix = 'Complete', length = 50)
    for (inputMatrix_batch, inputLabels_batch), outputLabels_batch in data_gen:
      #  tracker_outputs = model.predict([inputMatrix_batch, inputLabels_batch])
        tracker_outputs = getOutputTensor(model, inputMatrix_batch, inputLabels_batch)
        samples = drawSamples(model, episodes, sigma, outputLabels_batch)
        final_gradients, reward = getFinalGradients(model, samples, outputLabels_batch)
        model = getUpdatedModelWithGradients(model, final_gradients)
        i += 1
        if verbose == 1: printProgressBar(i, steps_per_epoch, prefix = epochCount, suffix = 'Complete, reward: %f' % 0, length = 50)
    return model

In [79]:
def reinforceModel(model, data_gen, episodes, sigma, steps_per_epoch, epochs, verbose=1):
    for e in range(epochs):
        epochCount = 'Epoch %d/%d:' % (e+1, epochs)
        model = reinforceModelForEpoch(model, data_gen, episodes, sigma, steps_per_epoch, epochCount, verbose)
    return model

In [80]:
for inputMatrix, labels in [set_gen0]:
        labels = labels[:, output_begin:output_begin+num_outputs]
        inputMatrix, inputLabels, outputLabels = getSequencesToSequences(inputMatrix, labels, timesteps) 
        data_gen = combined_generator2(inputMatrix, inputLabels, outputLabels, timesteps, batch_size)
        steps_per_epoch = ((inputMatrix.shape[0]/timesteps)-1)/batch_size
        reinforceModel(full_model, data_gen, 5, 0.02, steps_per_epoch, 1, verbose=1)

(1, 10, 3) <class 'tuple'>------------------------------------| 0.0% Complete
(1, 10, 3) <class 'tuple'>------------------------------------| 1.2% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 2.5% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 3.8% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 5.0% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 6.2% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 7.5% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 8.8% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 10.0% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------------| 11.2% Complete, reward: 0.000000
(1, 10, 3) <class 'tuple'>------------------------------

KeyboardInterrupt: 

In [41]:
set_gen = readBIWIDataset(subjectList = trainingSubjects, preprocess_input = preprocess_input)
set_gen0 = next(set_gen)

All frames and annotations from 1 datasets have been read by 2019-02-20 03:36:26.258004
